# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

#### Importamos las librerías necesarias

In [50]:
import lancedb
import numpy as np
import pyarrow as pa
import pandas as pd
from sentence_transformers import SentenceTransformer
from torchvision import models, transforms
from PIL import Image
import torch
from torchvision.models import ResNet18_Weights

#### Creamos la tabla mis_vectores 

In [23]:
# Configurar LanceDB
uri = "../../notebooks/data/ANN_prueba"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear 10,000 vectores de muestra aleatorios
np.random.seed(42)  # Para reproducibilidad
data = [
    {"vector": row, "item": f"item {i}"}
    for i, row in enumerate(np.random.random((19_999_0, 1536)).astype("float32"))
]

# Crear tabla en la base de datos
tbl = db.create_table("mis_vectores", data=data)
print(f"Tabla 'mis_vectores' creada con {len(data)} vectores.")
# Mostrar las primeras 5 filas de la tabla

Tabla 'mis_vectores' creada con 199990 vectores.


#### Resolución de task 1

In [29]:
import numpy as np

# 1. Generar un vector aleatorio con la misma dimensión que los embeddings en la tabla
query_vector = np.random.random(1536).astype("float32")

# 2. Realizar una búsqueda para encontrar los 5 elementos más cercanos
results = tbl.search(query_vector).limit(5).to_pandas()

# 3. Proyectar los resultados para mostrar solo las columnas "item" y "_distance"
results = results[["item", "_distance"]]

# 4. Excluir los elementos cuyo nombre sea 'item 500'
filtered_results = results[results["item"] != "item 500"]

# Mostrar los resultados finales
print("Cinco elementos más cercanos al vector creado (excluyendo 'item 500'):")
print(filtered_results)


Cinco elementos más cercanos al vector creado (excluyendo 'item 500'):
          item   _distance
0   item 36651  209.631943
1  item 182329  216.523422
2  item 170639  216.776688
3  item 134102  217.227112
4  item 154076  217.503479


No entendí muy bien a que se refieren con "la misma dirección que los embeddings que están en la tabla ...". Supuse que era la misma dimensión.

Ahí se muestran los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno.

**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [39]:
# Configurar LanceDB
uri = "../../notebooks/data/ANN_prueba1"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# 1. Definir el esquema utilizando pyarrow
schema = pa.schema([
    ("vector", pa.list_(pa.float32(), 4)),  # Vector de 4 dimensiones
    ("nombre", pa.string()),               # Columna de texto
    ("categoria", pa.string())             # Columna de texto
])

# 2. Crear una tabla vacía llamada "nueva_tabla"
nueva_tabla = db.create_table("nueva_tabla", schema=schema)
print("Tabla 'nueva_tabla' creada con el esquema definido.")

# 3. Insertar 5 registros en la tabla al crear un `pyarrow.Table`
data = [
    {"vector": [1.0, 2.0, 3.0, 4.0], "nombre": "item1", "categoria": "A"},
    {"vector": [4.0, 3.0, 2.0, 1.0], "nombre": "item2", "categoria": "B"},
    {"vector": [1.5, 2.5, 3.5, 4.5], "nombre": "item3", "categoria": "A"},
    {"vector": [4.5, 3.5, 2.5, 1.5], "nombre": "item4", "categoria": "B"},
    {"vector": [2.0, 3.0, 4.0, 5.0], "nombre": "item5", "categoria": "C"},
]

# Convertir los datos a un pyarrow.Table
arrow_table = pa.Table.from_pylist(data, schema=schema)

# Agregar datos a la tabla
nueva_tabla.add(arrow_table)
print("5 registros insertados en 'nueva_tabla'.")

# 4. Mostrar el contenido de la tabla
print("Contenido de 'nueva_tabla':")
print(nueva_tabla.to_pandas())


Tabla 'nueva_tabla' creada con el esquema definido.
5 registros insertados en 'nueva_tabla'.
Contenido de 'nueva_tabla':
                 vector nombre categoria
0  [1.0, 2.0, 3.0, 4.0]  item1         A
1  [4.0, 3.0, 2.0, 1.0]  item2         B
2  [1.5, 2.5, 3.5, 4.5]  item3         A
3  [4.5, 3.5, 2.5, 1.5]  item4         B
4  [2.0, 3.0, 4.0, 5.0]  item5         C


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [45]:
if "actualizacion_vectores" in db.table_names():
    db.drop_table("actualizacion_vectores")


In [46]:
# Configurar LanceDB
uri = "../../notebooks/data/ANN_prueba2"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# 1. Crear una tabla utilizando un DataFrame de Pandas
# Crear el DataFrame inicial
df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "vector": [[1.1, 2.1, 3.1], [4.15, 5.15, 6.15], [7.22, 8.22, 9.22], [10.1, 11.33, 12.66], [13.01, 14.02, 15.01]]
})

# Para convertir a una tabla de LanceDB hacemos el flow que recomendaron (Pandas -> PyArrow -> LanceDB)
# Convertir el DataFrame a un PyArrow Table
arrow_table = pa.Table.from_pandas(df)

# Crear la tabla en LanceDB
tabla = db.create_table("actualizacion_vectores", data=arrow_table)
print("Tabla creada con los datos iniciales.")

# 2. Actualizar el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
tabla.update(where="id = 3", values={"vector": [10.0, 11.0, 10.0]})
print("Vector actualizado para la fila con id=3.")

# 3. Filtrar la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0
# Convertir la tabla LanceDB a un DataFrame de pandas
df_actualizada = tabla.to_pandas()

# Filtrar las filas donde al menos un valor del vector sea mayor a 9.0
df_filtrado = df_actualizada[df_actualizada["vector"].apply(lambda x: any(v > 9.0 for v in x))]

# Mostrar los resultados
print("Filas donde al menos un valor del vector es mayor a 9.0:")
print(df_filtrado)

Tabla creada con los datos iniciales.
Vector actualizado para la fila con id=3.
Filas donde al menos un valor del vector es mayor a 9.0:
   id                 vector
2   4   [10.1, 11.33, 12.66]
3   5  [13.01, 14.02, 15.01]
4   3     [10.0, 11.0, 10.0]


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [62]:
# Configurar LanceDB
uri = "../../notebooks/data/ANN_multimodal"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Modelos para embeddings
text_model = SentenceTransformer('all-MiniLM-L6-v2')  # Modelo para embeddings de texto
image_model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # Modelo ResNet18 para embeddings de imágenes
image_model.fc = torch.nn.Identity()  # Usar la capa antes de la clasificación final
image_model.eval()  # Configurar el modelo en modo evaluación

# Transformaciones para las imágenes
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para generar embeddings de texto
def get_text_embedding(text):
    return text_model.encode(text).tolist()

# Función para generar embeddings de imágenes
def get_image_embedding(image_path):
    image = Image.open(image_path).convert('RGB')  # Cargar la imagen
    image_tensor = image_transform(image).unsqueeze(0)  # Aplicar transformaciones
    with torch.no_grad():
        embedding = image_model(image_tensor).squeeze(0).numpy()
    return embedding.tolist()

# 1. Crear la tabla con datos multimodales
# Ejemplo de datos
data = [
    {
        "texto": "El océano es vasto y azul.",
        "imagen": "ocean.jpg",
        "embedding_texto": get_text_embedding("El océano es vasto y azul."),
        "embedding_imagen": get_image_embedding("../../notebooks/data/images/ocean.jpg")
    },
    {
        "texto": "El cielo despejado tiene un tono azul intenso.",
        "imagen": "sky.jpg",
        "embedding_texto": get_text_embedding("El cielo despejado tiene un tono azul intenso."),
        "embedding_imagen": get_image_embedding("../../notebooks/data/images/sky.jpg")
    },
    {
        "texto": "Un bosque verde y frondoso.",
        "imagen": "forest.jpg",
        "embedding_texto": get_text_embedding("Un bosque verde y frondoso."),
        "embedding_imagen": get_image_embedding("../../notebooks/data/images/forest.jpg")
    }
]

# Convertir los datos a un DataFrame
df = pd.DataFrame(data)

# Crear la tabla en LanceDB
tabla = db.create_table("multimodal_table", data=df)
print("Tabla multimodal creada con los datos iniciales.")

# 2. Consulta combinada de texto e imagen
query_text_1 = "La tecnología avanza rápido"  # Texto de consulta
query_image_path_1 = "../../notebooks/data/images/blue.jpg"  # Ruta de la imagen de consulta (predominantemente azul)

# Generar embedding de texto
query_text_embedding_1 = text_model.encode(query_text_1).astype(np.float32)

# Generar embedding de imagen desde la ruta
query_image_1 = Image.open(query_image_path_1).convert("RGB")  # Cargar la imagen
query_image_tensor_1 = image_transform(query_image_1).unsqueeze(0)
with torch.no_grad():
    query_image_embedding_1 = image_model(query_image_tensor_1).squeeze().numpy().astype(np.float32)

# Convertir la tabla multimodal de LanceDB a pandas para combinar búsquedas
df_table = tabla.to_pandas()

# Calcular las distancias de texto y de imagen para cada fila
df_table["distance_text"] = df_table["embedding_texto"].apply(
    lambda x: np.linalg.norm(np.array(x) - query_text_embedding_1)
)
df_table["distance_image"] = df_table["embedding_imagen"].apply(
    lambda x: np.linalg.norm(np.array(x) - query_image_embedding_1)
)

# Calcular la distancia promedio entre texto e imagen
df_table["distance_avg"] = (df_table["distance_text"] + df_table["distance_image"]) / 2

# Ordenar por menor distancia promedio
resultados_combinados = df_table.sort_values(by="distance_avg")[["texto", "imagen", "distance_avg"]]

# 3. Mostrar resultados combinados
print("Resultados de búsqueda combinada:")
print(resultados_combinados)


Tabla multimodal creada con los datos iniciales.
Resultados de búsqueda combinada:
                                            texto      imagen  distance_avg
1  El cielo despejado tiene un tono azul intenso.     sky.jpg      8.517910
0                      El océano es vasto y azul.   ocean.jpg      8.539754
2                     Un bosque verde y frondoso.  forest.jpg     10.236463


Ahí están los resultados, lo hice con numpy y pandas al final en vez de con `search()` porque me salía un error en mi local.